In [13]:
from ipyleaflet import Map, basemaps, DrawControl, GeoData
import geopandas as gpd
from shapely.geometry import Polygon, box
from ipywidgets import Button, Output
from IPython.display import display
import numpy as np

# Create a map
m = Map(center=(51.5, -0.1), zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)

# Create a draw control
draw_control = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}})
m.add_control(draw_control)

# Display the map
display(m)

# Create an output widget to display results
output = Output()
display(output)

# Function to execute when a new shape is drawn
def handle_draw(self, action, geo_json):
    # Get the coordinates of the polygon
    coords = geo_json['geometry']['coordinates'][0]
    polygon = Polygon(coords)

    # Define the size of your square sections (in degrees)
    square_size = 0.1

    # Create a grid of squares within your area of interest
    squares = []
    for x in np.arange(min([coord[0] for coord in coords]), max([coord[0] for coord in coords]), square_size):
        for y in np.arange(min([coord[1] for coord in coords]), max([coord[1] for coord in coords]), square_size):
            square = box(x, y, x+square_size, y+square_size)
            if polygon.intersects(square):
                squares.append(square)

    # Create a GeoDataFrame for the squares
    squares_gdf = gpd.GeoDataFrame(gpd.GeoSeries(squares), columns=['geometry'])

    # Display the squares on the map
    m.add_layer(GeoData(geo_dataframe=squares_gdf))

    # Output the lat/lon coordinates of each square section
    with output:
        for i, row in squares_gdf.iterrows():
            minx, miny, maxx, maxy = row.geometry.bounds
            print(f'Square {i}:')
            print(f'  Bottom left: ({miny}, {minx})')
            print(f'  Top right: ({maxy}, {maxx})')

# Attach the function to the draw control
draw_control.on_draw(handle_draw)


Map(center=[51.5, -0.1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

Output()